In [1]:
import json
import os
import numpy as np
from scipy.spatial import ConvexHull
from UliEngineering.Math.Coordinates import BoundingBox
import xml.etree.cElementTree as ET
from PIL import Image
from glob import glob

In [7]:
path_to_images = 'D:/EveryFuckingProgram_home/tf_train/images/' #INSERT HERE A PATH TO YOUR OWN DATASET >>>
path_to_annotations = 'D:/EveryFuckingProgram_home/tf_train/annotations/'

In [3]:
def get_height_and_width(cur_filename):
    im = Image.open(cur_filename)
    return im.size

In [4]:
def BB_8(data):
    pts = []
    for i in range(len(data['hand_pts'])):
        pts.append(data['hand_pts'][i][0:2])
    hull_js = ConvexHull(pts)
    hull_points = []
    for i in hull_js.vertices:
        hull_points.append(hull_js.points[i])
    the_hull = []
    for i in range(len(hull_points)):
        the_hull.append(hull_points[i].tolist())
    the_hull_array = np.asarray(the_hull)

    cur_bb = BoundingBox(the_hull_array)
    eps = 0 #int(cur_bb.area/100)
    
    return max(int(cur_bb.minx) - eps, 0), max(int(cur_bb.miny) - eps,0), min(int(cur_bb.maxx) + eps, cur_width), min(int(cur_bb.maxy) + eps, cur_height)

In [6]:
def create_xml(path_to_annotations, img_file, cur_filename, cur_width, cur_height, xmin, ymin, xmax, ymax):
        
    annotation = ET.Element('annotation', verified = "yes")
    folder = ET.SubElement(annotation, "folder").text = "images"
    filename = ET.SubElement(annotation, "filename").text = img_file
    path= ET.SubElement(annotation, "path").text = cur_filename
    source = ET.SubElement(annotation, "source")

    database = ET.SubElement(source, "database").text = 'Faradenza_DB'


    size = ET.SubElement(annotation, "size")

    width = ET.SubElement(size, "width").text = str(cur_width)
    height = ET.SubElement(size, "height").text = str(cur_height)
    depth = ET.SubElement(size, "depth").text = '3'

    segmented = ET.SubElement(annotation, "segmented").text = '0'
    object = ET.SubElement(annotation, "object")

    name = ET.SubElement(object, "name").text = 'hand'
    pose = ET.SubElement(object, "pose").text = 'Unspecified'
    truncated = ET.SubElement(object, "truncated").text = '0'
    difficult = ET.SubElement(object, "difficult").text = '0'
    bndbox = ET.SubElement(object, "bndbox")

    xmin = ET.SubElement(bndbox, "xmin").text = str(xmin)
    ymin = ET.SubElement(bndbox, "ymin").text = str(ymin)
    xmax = ET.SubElement(bndbox, "xmax").text = str(xmax)
    ymax = ET.SubElement(bndbox, "ymax").text = str(ymax)


    tree = ET.ElementTree(annotation)
    tree.write(path_to_annotations+os.path.splitext(img_file)[0] +'.xml')

In [8]:
for cur_json_name in glob(path_to_images+ '/*.json'):
    with open(cur_json_name, 'r') as file:
        data = json.load(file)
        
    img_file = os.path.basename(data['img_path'])
    cur_filename = os.path.abspath(path_to_images) + img_file
    
    cur_width, cur_height = get_height_and_width(cur_filename)
    
    xmin, ymin, xmax, ymax = BB_8(data)
    
    create_xml(path_to_annotations, img_file, cur_filename, cur_width, cur_height, xmin, ymin, xmax, ymax)